In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.linear_model as sk
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from random import randint
from IPython.display import display
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
data = pd.read_csv("haberman.data",header=None)
data_df = pd.DataFrame(data)
data_df.dropna(inplace=True)
data_df.columns = ['Age','Year','Nodes','Survival_status']
data_df['Survival_status'] = data_df['Survival_status'].astype(int)
data_df['Survival_status'] = data_df['Survival_status'].map({2:0 , 1:1})
data_df.drop_duplicates(inplace=True)

In [ ]:
def class_percentage(p,n):
    a,b = list(p['Survival_status']).count(1),list(p['Survival_status']).count(0)
    return a*100/n,b*100/n

In [ ]:
class0_df = data_df[data_df['Survival_status']!=1]
class0_df.reset_index(drop=True)
class1_df = data_df[data_df['Survival_status']!=0]
class1_df.reset_index(drop=True)
print()

In [ ]:
class0_df['Survival_status'].value_counts()

In [ ]:
original_data = data_df.copy(deep=True)
original_data_class0 = class0_df.copy(deep=True)
original_data_class1 = class1_df.copy(deep=True)    
class0_df.head(4)

In [ ]:
def flip(a):
    n = len(a)
    p = int(n*0.9)
    for i in range(p):
        rand = randint(0,n-1)
        a['Survival_status'][a.index[rand]] = int(not a['Survival_status'][a.index[rand]])

In [ ]:
flip(class0_df)
print(class_percentage(class0_df,len(class0_df)))

In [ ]:
flip(class1_df)
print(class_percentage(class1_df,len(class1_df)))

# OriginalData Train test split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(data_df.loc[:,data_df.columns!='Survival_status'],data_df['Survival_status'],test_size = 0.38)

# Mislabeled Data 

In [ ]:
Mislabeled = class0_df.add(class1_df,fill_value=0)

In [ ]:
data_df['Survival_status'].value_counts()

In [ ]:
Mislabeled['Survival_status'].value_counts()

In [ ]:
print(177/306 , 193/306 , 48/306)

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(Mislabeled.loc[:,Mislabeled.columns!='Survival_status'],Mislabeled['Survival_status'],test_size = 0.38)

In [ ]:
Y_train.unique()

# Model Training

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,Y_train)
predict = rf.predict(x_test)
accuracy_score(y_test,predict)

# Original_Model

In [ ]:
rf_original = RandomForestClassifier()
rf_original.fit(x_train,y_train)
predict_ori = rf_original.predict(x_test)
accuracy_score(y_test,predict_ori)

In [ ]:
data_df.describe()

# Clustering

In [ ]:
cl = KMeans(n_clusters=2)
cl.fit(Mislabeled.loc[:,Mislabeled.columns!='Survival_status'])
predict = cl.labels_

In [ ]:
print(list(predict).count(0)*100/289,list(predict).count(1)*100/289)
Mislabeled.reset_index(drop=True,inplace=True)

In [ ]:
for i in range(len(Mislabeled)):
    if predict[i]!=Mislabeled['Survival_status'][i]:
        Mislabeled['Survival_status'][i] = np.NaN        

In [ ]:
Mislabeled['Survival_status'].value_counts()

In [ ]:
Mislabeled.dropna(inplace=True)

In [ ]:
Mislabeled.shape

In [ ]:
x1,x2,y1,y2 = train_test_split(Mislabeled.loc[:,Mislabeled.columns!='Survival_status'],Mislabeled['Survival_status'],test_size = 0.38)

In [ ]:
rf_final = RandomForestClassifier()
rf_final.fit(x1,y1)
p1 = rf_final.predict(x_test)
accuracy_score(y_test,p1)

# Plot 3D Graph

In [ ]:
%matplotlib qt  
def plot_3d(x,y,z,label):
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    colour = label.map({0:'r',1:'g'})
    ax.scatter(x,y,z,c = colour)
plot_3d(data_df['Age'],data_df['Year'],data_df['Nodes'],data_df['Survival_status'])
plot_3d(Mislabeled['Age'],Mislabeled['Year'],Mislabeled['Nodes'],Mislabeled['Survival_status'])